In [19]:
import logging

from Loader import fileloader_proto as fl
from Loader import datahandledatabase as dhdb
from Loader import datacompute as dc
from Loader import datapreparation as dp
from Loader import dfutils as utils
import pandas as pd
import numpy as np
import posixpath
from datetime import datetime
import time
import xlwings as xw
import copy

logger = logging.getLogger("")
logger.setLevel(logging.INFO)

**Ingresos Nuevos**
----------
Ejecutar el siguiente codigo como requisito para las siguientes secciones de Ingresos Nuevos

In [20]:
month="201907"
inifile = fl.ReadIniFile(mercado="empresas")
parser = inifile.getIniFileParser()

# Configurando los archivos de configuración para administrar la base de datos
dbparser = inifile.getDbIniFileParser()
dbmanager = dhdb.DbDataProcess(month)
dbmanager.setParser(dbparser)


INFO:    Values loaded from globals.ini:
INFO:     * mainpath C:/Users/jsaenza/OneDrive - Entel Peru S.A
INFO:     * scriptspath C:/Users/jsaenza/Documents/Comisiones/Comisiones-python
INFO:     * databasepath C:/Users/jsaenza/OneDrive - Entel Peru S.A/SQLiteDB
INFO:    datapath value is C:/Users/jsaenza/OneDrive - Entel Peru S.A/MercadoEmpresas/Data Fuente Comisiones/xlsx
INFO:    testpath value is C:/Users/jsaenza/OneDrive - Entel Peru S.A/MercadoEmpresas/Data Fuente Comisiones/test
INFO:    Setting database path to C:/Users/jsaenza/OneDrive - Entel Peru S.A/SQLiteDB/mercado_empresas_db.sqlite


***<span style="color:red">Copiar</span>*** Ingresos Nuevos del Archivo de Atracción
-----

Copia los ingresos nuevos desde el archivo enviado por atracción hacia el archivo de Ingresos Nuevos de comisiones.

**Requisitos**: 
- Haber filtrado las personas de interes en el archivo que envia attracion
- Asegurarse la hoja se llame BASE en el archivo de ingresos nuevos de comisiones
- Asegurarse que los nombres de columnas del archivo de atraccion no hayan cambiado

**Pasos**:
- En el archivo de atracción filtrar solo las personas que comisionan, eliminar las demás
- Ejecutar script
- Convertir la columna DNI a formato Texto
- Jalar las fórmulas
- Proseguir con la subida a la base de datos de los nuevos ingresos





In [10]:
#Obtenemos la lista de los nombres de los libros de Excel abiertos.
wb_names = utils.getOpenedWorkbooksNames()
#Seleccionamos el archivo que contenga "REPORTEREMUNERACIONES"
keyname = "REPORTEREMUNERACIONES"
atraccion_ingresos_filename = [s for s in wb_names if keyname in s][0]
#Nos conectamos al archivo y hoja de calculo enviado por atraccion
wb_ingresos = xw.Book(atraccion_ingresos_filename)
sht = wb_ingresos.sheets[0]
#Nos posicionamos en la celda A1 extendemos hacia la esquina opuesta retornando el contenido como dataframe
df = sht.range('A1').options(pd.DataFrame,
                         index=False, expand='table').value
#Filtramos las columnas que debe tener el archivo de atraccion.
cols_required = cols_required[:-3] + ['Variable', 'VP', 'CC', 'Jefe DIrecto']
#Obtenemos el dataframe a ser pegado en nuestro archivo de comisiones.
df_final = df[cols_required]

# Cargamos el sheet de Ingresos porque para que xlwings abra el archivo necesita el nombre
# del fichero y también necesitamos las columnas definidas en el archivo de configuracion
section_ingresos = fl.SectionObj(inifile,"Ingresos")
# Guardamos el nombre de nuestro archivo de nuevos ingresos
comis_ingresos_archivo_nombre = section_ingresos.getParameter('filelist')[0]
# Obtenemos las columnas que deberia tener el reporte de ingresos nuevos.
cols_required = section_ingresos.getParameter('cols')

#Nos conectamos a nuestro archivo de Ingresos
wb_comis_ingresos = xw.Book(comis_ingresos_archivo_nombre)
sheet_comis_ingresos = wb_comis_ingresos.sheets('BASE')
#Obtener el nro. de fila del último registro
lastrow = sheet_comis_ingresos.api.Cells(65536, 1).End(xw.constants.Direction.xlUp).Row
#Pegamos el dataframe
sheet_comis_ingresos.range((lastrow+1, 1)).options(header=False, index=False).value = df_final


Insertar ***<span style="color:red">Nuevos</span>*** Ingresos en la BD
-----

Inserta los registros mracados con ADD_TO_DB = "SI" del dataframe de Ingresos de comisiones a la base de datos

In [5]:
section_ingresos = fl.SectionObj(inifile,"Ingresos")
ingresos = fl.LoadFileProcess(section_ingresos).loadFile()
section_ingresos_cdp = fl.SectionObj(inifile,"IngresosCDP")
ingresos_cdp = fl.LoadFileProcess(section_ingresos_cdp).loadFile()

# Separamos los que se van a agregar a la base de datos y los que unicamente se va actualizar su CODIGO_INAR
ingresos_add = ingresos[ingresos.ADD_TO_DB == "SI"]
ingresos_add.drop(['ADD_TO_DB','UPDATE_CODIGO_INAR'], axis=1, inplace=True)

ingresos_add_CDP = ingresos_cdp[ingresos_cdp.ADD_TO_DB == "SI"]
ingresos_add_CDP.drop(['ADD_TO_DB','UPDATE_PUESTO','UPDATE_CODIGO_INAR'], axis=1, inplace=True)

El tamaño de Ingresos es 94 registros
El tamaño de IngresosCDP es 69 registros


C:\Users\jsaenza\Anaconda3\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


**Ejecutar lo siguiente para modificar la BD**

In [6]:
dbmanager.dbOperation('insert','tblEmpleados',ingresos_add)
dbmanager.dbOperation('insert','tblEmpleados',ingresos_add_CDP)

Los registros de la tabla tblEmpleados es 0 registros insertados 
Los registros de la tabla tblEmpleados es 0 registros insertados 


Actualizar ***<span style="color:red">Codigo Inar</span>*** Ingresos en la BD
----

Actualiza los registros marcados con UPDATE_CORDIGO_INAR = "SI" del dataframe de ingresos a la Base de Datos.

In [21]:
section_ingresos = fl.SectionObj(inifile,"Ingresos")
ingresos = fl.LoadFileProcess(section_ingresos).loadFile()
section_ingresos_cdp = fl.SectionObj(inifile,"IngresosCDP")
ingresos_cdp = fl.LoadFileProcess(section_ingresos_cdp).loadFile()

ingresos_update = ingresos[ingresos.UPDATE_CODIGO_INAR == "SI"]
ingresos_update.drop(['ADD_TO_DB','UPDATE_CODIGO_INAR'], axis=1, inplace=True)

ingresos_update_CDP = ingresos_cdp[ingresos_cdp.UPDATE_CODIGO_INAR == "SI"]
ingresos_update_CDP.drop(['ADD_TO_DB','UPDATE_CODIGO_INAR','UPDATE_PUESTO'], axis=1, inplace=True)
ingresos_update_CDP

El tamaño de Ingresos es 94 registros
El tamaño de IngresosCDP es 69 registros


C:\Users\jsaenza\Anaconda3\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,DNI,NOMBRES,POSICION_EMPL,FECHA_INGRESO,CODIGO_INAR
13,40712693,MORALES OLAZABAL JAVIER ALONSO,CONSULTOR DE SERVICIO VALOR AGREGADO,2019-06-01,JMORALES
14,41032819,LAGUNA DIAZ VICTOR ROMAN,EJECUTIVO DE REGIONES,2019-06-01,VLAGUNAD
32,45487775,GUILLÉN RENGIFO JOSÉ LUIS,EJECUTIVO DE DESARROLLO PYME,2019-07-01,JGUILLEN
43,41179299,DIAZ AMAYO FERNANDO ALONSO,EJECUTIVO DE DESARROLLO PYME REGIONES,2019-07-01,FDIAZ5


**Ejecutar lo siguiente para modificar la BD**

In [22]:
dbmanager.dbOperation('update','UpdateEmpleadoCodigoInar',ingresos_update)
dbmanager.dbOperation('update','UpdateEmpleadoCodigoInar',ingresos_update_CDP)

Los registros de la tabla UpdateEmpleadoCodigoInar es 0 registros actualizados 
Los registros de la tabla UpdateEmpleadoCodigoInar es 4 registros actualizados 


Actualizar ***<span style="color:red">Puestos</span>*** Ingresos en la BD
-----

Actualiza los puestos por DNI ingresados en el archivo de Ingresos de comisiones pestaña "UpdatePuestos" hacia la Base de Datos.

In [15]:
section_ingresos_cdp = fl.SectionObj(inifile,"IngresosCDP")
ingresos_cdp = fl.LoadFileProcess(section_ingresos_cdp).loadFile()

ingresos_update_puesto_CDP = ingresos_cdp[ingresos_cdp.UPDATE_PUESTO == "SI"]
ingresos_update_puesto_CDP.drop(['ADD_TO_DB','UPDATE_CODIGO_INAR','UPDATE_PUESTO'], axis=1, inplace=True)

El tamaño de IngresosCDP es 69 registros


C:\Users\jsaenza\Anaconda3\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


**Ejecutar lo siguiente para modificar la BD**

In [18]:
dbmanager.dbOperation('update','UpdatePuesto',ingresos_update_puesto_CDP)

Los registros de la tabla UpdatePuesto es 37 registros actualizados 


**Ejecutar lo siguiente para modificar la BD**

In [17]:
len(ingresos_update_puesto_CDP)

37